In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read in CSV
train_pd = pd.read_csv('train.csv')
test_pd = pd.read_csv('test.csv')

# Convert string representation of list to actual list of floats
train_pd['features'] = train_pd['features'].apply(lambda x: [float(i) for i in x.strip('[]').split(',')])
test_pd['features'] = test_pd['features'].apply(lambda x: [float(i) for i in x.strip('[]').split(',')])

x_train = torch.from_numpy(np.array(train_pd['features'].values.tolist(), np.float32))
y_train = torch.from_numpy(np.array(train_pd['outcome'].values.tolist(), np.int64))

x_test = torch.from_numpy(np.array(test_pd['features'].values.tolist(), np.float32))
y_test = torch.from_numpy(np.array(test_pd['outcome'].values.tolist(), np.int64))

# Make sure the data is zero-indexed
y_train = y_train - 1
y_test = y_test - 1
print(y_train.unique())

tensor([0, 1, 2, 3, 4])


In [3]:
print(x_train.shape, y_train.shape)

torch.Size([125973, 113]) torch.Size([125973])


In [4]:
from torch.utils.data import Dataset, DataLoader
class MyDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __len__(self):
        return self.x.shape[0]
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]
    
train_dataset = MyDataset(x_train, y_train)
test_dataset = MyDataset(x_test, y_test)

In [5]:
class myNeuralNet(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.ln1 = nn.Linear(input_size, 128)
        self.ln2 = nn.Linear(128, 64)
        self.ln3 = nn.Linear(64, 32)
        self.ln4 = nn.Linear(32, 5)
    def forward(self, x):
        x = F.relu(self.ln1(x))
        x = F.relu(self.ln2(x))
        x = F.relu(self.ln3(x))
        x = self.ln4(x)
        x = F.softmax(x, dim=-1)
        return x

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
def training_loop(n_epochs, model, lr, batch_size):
    model = model.to(device)

    batch_size = batch_size
    num_epochs = n_epochs
    lr = lr
    opt = optim.Adam(model.parameters(), lr=lr)
    loss_fcn = nn.CrossEntropyLoss()

    train_loss = []
    test_loss = []
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode

        # Initialize variables to store the total loss for this epoch
        total_train_loss = 0
        total_test_loss = 0
        correct_predictions = 0
        total_predictions = 0
        # Training loop
        for batch_id, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)  # Move the data to the device that is used
            opt.zero_grad()  # Zero the gradients
            output = model(x)  # Forward pass
            loss = loss_fcn(output, y)  # Compute the loss

            total_train_loss += loss.item()  # Accumulate the loss
            _, predicted = torch.max(output.data, 1)  # Get the predicted classes
            total_predictions += y.size(0)
            correct_predictions += (predicted == y).sum().item()
            loss.backward()  # Backward pass
            opt.step()  # Update the parameters

        # Store the average training loss for this epoch
        train_loss.append(total_train_loss / len(train_loader))
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch + 1} - Train Accuracy: {correct_predictions / total_predictions}")
        # Validation loop
        model.eval()  # Set the model to evaluation mode
        correct_predictions = 0
        total_predictions = 0
        with torch.no_grad():  # No need to compute gradients during validation
            for batch_id, (x, y) in enumerate(test_loader):
                x, y = x.to(device), y.to(device)  # Move the data to the device that is used
                output = model(x)  # Forward pass
                loss = loss_fcn(output, y )  # Compute the loss
                total_test_loss += loss.item()  # Accumulate the loss
                _, predicted = torch.max(output.data, 1)
                total_predictions += y.size(0)
                correct_predictions += (predicted == y).sum().item()

        # Store the average validation loss for this epoch
        test_loss.append(total_test_loss / len(test_loader))

        # Print the losses every 5 epochs
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch + 1} - Train Loss: {train_loss[-1]}, Test Loss: {test_loss[-1]}")
            print(f"Epoch {epoch + 1} - Test Accuracy: {correct_predictions / total_predictions}")

        # Save the best model so far
        if epoch == 0 or test_loss[-1] < np.min(test_loss[:-1]):
            best_model = model
            best_epoch = epoch

        # Early stopping
        if epoch - best_epoch >= 30:
            print(f"Early stopping at epoch {epoch + 1}")
            break
    # Plot the training and validation losses
    import matplotlib.pyplot as plt
    plt.plot(train_loss, label="Train Loss")
    plt.plot(test_loss, label="Test Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.title("Training and Validation Losses")
    plt.show()

In [11]:
input_size = x_train.shape[1]
models = myNeuralNet(input_size)
training_loop(10, models, 0.004, 128)

Epoch 5 - Train Accuracy: 0.9747723718574616
Epoch 5 - Train Loss: 0.9300287121443579, Test Loss: 1.1809837797940788
Epoch 5 - Test Accuracy: 0.7229418026969482
Epoch 10 - Train Accuracy: 0.9740817476760893
Epoch 10 - Train Loss: 0.9307262445464352, Test Loss: 1.1771779585692843
Epoch 10 - Test Accuracy: 0.7267565649396736
